<a href="https://colab.research.google.com/github/dkapitan/scik/blob/master/notebooks/ggd-rijnmond-rotterdam-covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# make sure we're using pandas >= 1.0 and we need CBS OData
!pip install update pandas cbsodata

In [0]:
import requests
from zipfile import ZipFile
from io import BytesIO, StringIO

from bs4 import BeautifulSoup
import pandas as pd
import cbsodata


def get_gwb():
    """
    Retrieves data from CBS for mapping each address to buurt, wijk and gemeente from 2019.

    Returns:
      list of length 4 with dataframes [buurt, gemeente, mapping address --> gwb, wijken]

    Source: https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer
    Numeric columns are downcasted to uint16 to lower memory footprint
    """
    URL = 'https://www.cbs.nl/-/media/_excel/2019/42/2019-cbs-pc6huisnr20190801_buurt.zip'
    get_ = requests.get(URL)
    with ZipFile(BytesIO(get_.content)) as zipfile:
      dfs = [pd.read_csv(zipfile.open(file),
                          delimiter=';',
                          )
              for file in zipfile.namelist()
             ]
    for df in dfs:
      df.loc[:, df.select_dtypes('int64').columns] = df.select_dtypes('int64').astype('uint16')
    return dfs


def get_kerncijfers_gwb():
  """
  Retrieves Kerncijfers Wijken en Buurten from 2019
  """
  URL = 'https://www.cbs.nl/-/media/cbs/dossiers/nederland%20regionaal/wijk-en-buurtstatistieken/_exel/kwb-2019.xls'
  return pd.read_excel(URL)


def get_rivm_data():
  """
  Scrapes RIVM website for number of reported hospitalizations per gemeente.
  """
  RIVM = 'https://www.rivm.nl/actuele-informatie-over-coronavirus'
  rivm = requests.get(RIVM)
  soup = BeautifulSoup(rivm.content, 'html.parser')
  data = StringIO(soup.find(id='csvData').text)
  return pd.read_csv(data, sep=';', skip_blank_lines=1)


def get_regionale_indelingen():
  """
  Retrieves regionale indelingen from CBS.

  https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=84721NED&_theme=232
  """
  return pd.DataFrame(cbsodata.get_data('84721NED'))


def get_data_ggd_rijnmond():
  """ Retrieves data from GGD Rijnmond on number of positive tests"""
  URL = 'https://www.ggdrotterdamrijnmond.nl/nieuws/patienten-coronavirus-in/'
  df = pd.read_html(URL, header=0)[0].iloc[0:-1,:]
  df.iloc[:,-1] = pd.to_numeric(df.iloc[:,-1], downcast='unsigned')
  rename = {k: v for k,v in zip(df.columns, ('gemeente', 'aantal_positief_getest'))}
  return df.rename(columns=rename)

In [0]:
# load all data
buurt, gemeente, adres, wijk = get_gwb()
kerncijfers = get_kerncijfers_gwb()
kerncijfers_gemeente = kerncijfers.loc[kerncijfers.recs == 'Gemeente',:]
rivm = get_rivm_data()
gebieden = get_regionale_indelingen()
ggd010 = get_data_ggd_rijnmond()

In [122]:
df = (ggd010
      .replace({'Capelle a/d IJssel': 'Capelle aan den IJssel', 'Krimpen a/d IJssel': 'Krimpen aan den IJssel'})
      .join(rivm.set_index('Gemeente'), 'gemeente')
      .rename(columns={'Aantal': 'aantal_ziekenhuisopnames',
                       'BevAant': 'aantal_inwoners',
                       'Gemnr': 'gemeente_code',
                       'gemeente': 'gemeente_naam',
                       'Aantal per 100.000 inwoners': 'prevalentie_ziekenhuisopnames'})
      .assign(prevalentie_positief_getest=lambda df: (df.aantal_positief_getest/df.aantal_inwoners*10**5).astype('uint8'))
      .loc[:, ['gemeente_code',
          'gemeente_naam',
          'aantal_inwoners',
          'aantal_positief_getest',
          'aantal_ziekenhuisopnames',
          'prevalentie_positief_getest',
          'prevalentie_ziekenhuisopnames']]
      .sort_values('gemeente_naam')
)
df

,gemeente_code,gemeente_naam,aantal_inwoners,aantal_positief_getest,aantal_ziekenhuisopnames,prevalentie_positief_getest,prevalentie_ziekenhuisopnames
0,613,Albrandswaard,25589,35,5,136,19.5
1,489,Barendrecht,48713,41,21,84,43.1
2,501,Brielle,17282,19,7,109,40.5
3,502,Capelle aan den IJssel,67151,75,23,111,34.3
4,1924,Goeree-Overflakkee,50049,165,35,73,69.9
5,530,Hellevoetsluis,40158,29,8,72,19.9
6,542,Krimpen aan den IJssel,29533,26,4,88,13.5
7,1621,Lansingerland,62396,79,22,126,35.3
8,556,Maassluis,33216,24,10,72,30.1
9,1930,Nissewaard,85237,60,24,70,28.2


In [106]:
gemeente

,Gemcode2019,Gemeentenaam2019
0,3,Appingedam
1,10,Delfzijl
2,14,Groningen
3,24,Loppersum
4,34,Almere
...,...,...
350,1963,Hoeksche Waard
351,1966,Het Hogeland
352,1969,Westerkwartier
353,1970,Noardeast-Fryslân


In [58]:
gebieden.loc[:,['Code_14', 'Naam_15']].drop_duplicates()

,Code_14,Naam_15
0,GG0706,GGD Drenthe ...
1,GG3406,GGD Amsterdam ...
2,GG1413,GGD Noord- en Oost-Gelderland ...
3,GG7206,GGD Fryslân ...
4,GG4816,Dienst Gezondheid & Jeugd ZHZ ...
5,GG4607,GGD Rotterdam-Rijnmond ...
6,GG2707,GGD Hollands-Noorden ...
7,GG1106,GGD Regio Twente ...
8,GG2209,GGD Flevoland ...
9,GG4506,GGD Hollands-Midden ...


IndexError: ignored